# Запросы к удалённым ресурсам и агрегация данных

Документация к iTunes Search API:
https://developer.apple.com/library/archive/documentation/AudioVideo/Conceptual/iTuneSearchAPI/Searching.html#//apple_ref/doc/uid/TP40017632-CH5-SW1

В документации есть Search и Lookup запросы.

In [1]:
import requests, json, urllib.parse

In [2]:
base_url = 'https://itunes.apple.com/search'

artist = 'AC/DC'

params = {
    'term': artist,
    'limit': 200,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [3]:
# Все доступные ключи объекта
print(list(data[0].keys()))

['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [4]:
import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)

In [5]:
# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(list(data[0].keys()))

['artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']


In [7]:
# Уникальные названия артистов
print(set(map(lambda x: x['artistName'], data)))

{'AC/DC'}


In [6]:
# Фильтрация с целью оставить искомого артиста
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)

In [8]:
release_date_extractor = lambda x: x['releaseDate']

first_song = min(data, key=release_date_extractor)

print('First song:', first_song['trackName'], first_song['releaseDate'])

latest_song = max(data, key=release_date_extractor)

print('Last song:', latest_song['trackName'], latest_song['releaseDate'])

First song: She's Got Balls 1975-02-17 08:00:00+00:00
Last song: Wild Reputation 2020-11-13 12:00:00+00:00


In [9]:
# Функция переводящая миллисекунды в читаемый формат времени
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H{minutes}M{seconds}S"

In [10]:
time_extractor = lambda x: x['trackTimeMillis']
longest_song = max(data, key=time_extractor)

print('Longest song:', longest_song['trackName'], convert_millis(longest_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)

print('Shortest song:', shortest_song['trackName'], convert_millis(shortest_song['trackTimeMillis']))

Longest song: Jailbreak (Live) 0H14M42S
Shortest song: Bonny (Live) 0H1M3S


In [11]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

# Удаление переменной, действие опционально
del sorted_by_album

print(list(album_names.values()))

['Highway to Hell', "'74 Jailbreak - EP", 'The Razors Edge', 'Ballbreaker', 'Powerage', 'Back In Black', 'Fly On the Wall', 'Stiff Upper Lip', 'Black Ice', 'For Those About to Rock (We Salute You)', 'Let There Be Rock', 'High Voltage', 'Dirty Deeds Done Dirt Cheap', 'Live at River Plate', 'Who Made Who', 'Live', "Live (Collector's Edition)", 'Iron Man 2', 'Bonfire', 'Backtracks (Deluxe Edition)', 'The Collection', 'The Complete Collection', 'Rock or Bust', 'POWER UP']


In [12]:
album_songs_count_extractor = lambda x: len(x[1])

album_with_max_songs = max(album_groups.items(), key=album_songs_count_extractor)
print('Max album songs:', album_names[album_with_max_songs[0]], len(album_with_max_songs[1]))

album_with_min_songs = min(album_groups.items(), key=album_songs_count_extractor)
print('Min album songs:', album_names[album_with_min_songs[0]], len(album_with_min_songs[1]))

Max album songs: Live (Collector's Edition) 17
Min album songs: '74 Jailbreak - EP 1


In [13]:
album_durations = map(lambda x: (x[0], sum(map(time_extractor, x[1]))), album_groups.items())
album_durations = dict(album_durations)

album_with_max_duration = max(album_durations.items(), key=lambda x: x[1])
print('Max album duration:', album_names[album_with_max_duration[0]], convert_millis(album_with_max_duration[1]))

album_witn_min_duration = min(album_durations.items(), key=lambda x: x[1])
print('Min album duration:', album_names[album_witn_min_duration[0]], convert_millis(album_witn_min_duration[1]))

Max album duration: Live (Collector's Edition) 1H34M55S
Min album duration: Bonfire 0H3M30S
